In [ ]:
import librosa
import numpy as np
import os
import pandas as pd

classes = {
    0: 'blues', 1: 'classical', 2: 'country', 3: 'disco', 
    4: 'hiphop', 5: 'jazz', 6: 'metal', 7: 'pop', 
    8: 'reggae', 9: 'rock'
}

def extract_mfcc_features(y, sr=22050, n_mfcc=20):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    mfcc_features = {}
    for i in range(n_mfcc):
        mfcc_features[f'mfcc{i+1}_mean'] = np.mean(mfcc[i])
        mfcc_features[f'mfcc{i+1}_var'] = np.var(mfcc[i])
    return mfcc_features

def process_30sec_mfcc(base_path):
    all_features = []
    
    for class_id, genre in classes.items():
        genre_path = os.path.join(base_path, genre)
        print(f"Processing {genre} files...")
        
        for audio_file in os.listdir(genre_path):
            if audio_file.endswith('.wav'):
                file_path = os.path.join(genre_path, audio_file)
                if audio_file == 'jazz.00054.wav':  # skipping jazz.00054, it was such a problem
                    continue
                y, sr = librosa.load(file_path, sr=22050) # btw 22050 is the sr of the audio files in the dataset
                mfcc_features = extract_mfcc_features(y, sr)
                mfcc_features['label'] = genre
                all_features.append(mfcc_features)
    
    df = pd.DataFrame(all_features)
    df.to_csv('gtzan_30sec_mfcc_features.csv', index=False)
    return df

path = '/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original'
mfcc_df = process_30sec_mfcc(path)

In [ ]:
def extract_coeff_features(y, sr=22050):
    features = {}
    
    # Chroma Features
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    features['chroma_stft_mean'] = np.mean(chroma_stft)
    features['chroma_stft_var'] = np.var(chroma_stft)
    
    # RMS Energy
    rms = librosa.feature.rms(y=y)
    features['rms_mean'] = np.mean(rms)
    features['rms_var'] = np.var(rms)
    
    # Spectral Features
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    features['spectral_centroid_mean'] = np.mean(spectral_centroid)
    features['spectral_centroid_var'] = np.var(spectral_centroid)
    
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features['spectral_bandwidth_mean'] = np.mean(spectral_bandwidth)
    features['spectral_bandwidth_var'] = np.var(spectral_bandwidth)
    
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    features['rolloff_mean'] = np.mean(rolloff)
    features['rolloff_var'] = np.var(rolloff)
    
    # Zero Crossing Rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=y)
    features['zero_crossing_rate_mean'] = np.mean(zero_crossing_rate)
    features['zero_crossing_rate_var'] = np.var(zero_crossing_rate)
    
    # Harmonic Features
    harmony = librosa.effects.harmonic(y)
    features['harmony_mean'] = np.mean(harmony)
    features['harmony_var'] = np.var(harmony)
    
    # Perceptual Features
    perceptr = librosa.feature.spectral_flatness(y=y)
    features['perceptr_mean'] = np.mean(perceptr)
    features['perceptr_var'] = np.var(perceptr)
    
    # Tempo
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    features['tempo'] = tempo
    
    return features

def process_30sec_coeff(base_path):
    all_features = []
    
    for genre in classes.values():
        genre_path = os.path.join(base_path, genre)
        print(f"Processing {genre} files...")
        
        for audio_file in os.listdir(genre_path):
            if audio_file.endswith('.wav'):
                file_path = os.path.join(genre_path, audio_file)
                if audio_file == 'jazz.00054.wav':  # Skipping problematic file
                    continue
                
                y, sr = librosa.load(file_path, sr=22050)  
                coeff_features = extract_coeff_features(y, sr)
                coeff_features['label'] = genre
                all_features.append(coeff_features)
    
    df = pd.DataFrame(all_features)
    df.to_csv('gtzan_30sec_coeff_features.csv', index=False)
    return df

path = '/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original'
coeff_df = process_30sec_coeff(path)

In [ ]:
df = pd.concat([mfcc_df.drop(columns=['label']), coeff_df], axis=1)
df.to_csv("features_30_sec.csv", index=False)